In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/philippine-law/philippine_laws.parquet


In [18]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [22]:
! pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.2 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [23]:
! pip install chromadb -q
! pip install -q "google-genai==1.7.0" 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
google-cloud-bigtable 2.28.1 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 wh

In [24]:
! pip install --upgrade google-generativeai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.32.1 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
google-cloud-bigtable 2.28.1 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.


In [47]:
import os
import pandas as pd
import numpy as np
from typing import List, Tuple
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
from chromadb.config import Settings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from google import genai
from google.generativeai import types
from kaggle_secrets import UserSecretsClient

In [48]:
# =======================
# 1. Configure Gemini API
# =======================
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [49]:
# ================================
# 2. Gemini Embedding Function
# ================================
def embedding_function(texts: List[str]) -> List[List[float]]:
    embeddings = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents=texts,
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"))
    return embeddings["embedding"] if isinstance(texts, str) else embeddings["embeddings"]

In [50]:
# ===========================================
# 3. Load Data and Paragraph-wise Chunking
# ===========================================
def load_and_chunk_data(parquet_path: str = "/kaggle/input/philippine-law", max_chars: int = 2000):
    df = pd.read_parquet(parquet_path)
    texts = df["text"].tolist()

    def paragraphwise_chunking(text: str) -> List[str]:
        paragraphs = text.split("\n\n")
        chunks, current_chunk = [], ""
        for para in paragraphs:
            if len(current_chunk) + len(para) < max_chars:
                current_chunk += para + "\n\n"
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = para + "\n\n"
        if current_chunk:
            chunks.append(current_chunk.strip())
        return chunks

    chunked_text = []
    doc_idx = []

    for i, text in enumerate(texts):
        chunks = paragraphwise_chunking(text)
        chunked_text.extend(chunks)
        doc_idx.extend([i] * len(chunks))

    metadatas_raw = df[["citation_information", "url", "label"]].to_dict(orient="records")
    metadatas_final = []
    for i in doc_idx:
        meta = metadatas_raw[i].copy()
        meta["doc_idx"] = i
        metadatas_final.append(meta)

    return chunked_text, metadatas_final

In [51]:
# ================================
# 4. Set Up and Populate ChromaDB
# ================================
def batch_add(collection, documents, embeddings, metadatas, ids, batch_size=5000):
    for i in range(0, len(documents), batch_size):
        collection.add(
            documents=documents[i:i + batch_size],
            embeddings=embeddings[i:i + batch_size],
            metadatas=metadatas[i:i + batch_size],
            ids=ids[i:i + batch_size]
        )

def setup_vector_db(chunked_text: List[str], metadatas: List[dict], path="./chroma_db"):
    local_model = SentenceTransformer("all-MiniLM-L6-v2")
    chroma_client = PersistentClient(path=path)
    collection = chroma_client.get_or_create_collection(name="philippine_omnicorpus_local")

    local_embeddings = local_model.encode(chunked_text, show_progress_bar=True, convert_to_numpy=True)
    ids = [f"doc_{i}" for i in range(len(chunked_text))]

    batch_add(collection, chunked_text, local_embeddings.tolist(), metadatas, ids)
    return collection, local_model

In [52]:
# ============================
# 5. Hybrid Search + Re-Rank
# ============================
def hybrid_search(query: str, collection, local_model, top_k: int = 10):
    q_emb = local_model.encode([query], convert_to_numpy=True).tolist()
    results = collection.query(
        query_embeddings=q_emb,
        n_results=top_k,
        include=["documents", "metadatas"]
    )
    return results["documents"][0], results["metadatas"][0]

def rerank_chunks(query: str, candidate_chunks: List[str], candidate_metas: List[dict],
                  local_model, fusion_weight: float = 0.5, top_k: int = 10):

    local_query_emb = normalize(local_model.encode([query], convert_to_numpy=True))
    local_chunk_embs = normalize(local_model.encode(candidate_chunks, convert_to_numpy=True))

    gemini_query_emb = normalize(np.array(embedding_function(query)).reshape(1, -1))
    gemini_chunk_embs = normalize(np.array(embedding_function(candidate_chunks)))

    local_scores = cosine_similarity(local_query_emb, local_chunk_embs)[0]
    gemini_scores = cosine_similarity(gemini_query_emb, gemini_chunk_embs)[0]

    combined_scores = fusion_weight * local_scores + (1 - fusion_weight) * gemini_scores
    sorted_indices = np.argsort(combined_scores)[::-1][:top_k]
    return [(candidate_chunks[i], candidate_metas[i], combined_scores[i]) for i in sorted_indices]

In [53]:
# ===========================
# 6. Generate Legal Response
# ===========================
def generate_legal_draft(query: str, context_chunks: List[Tuple[str, dict, float]], top_k: int = 3):
    top_context = "\n\n".join([chunk[0] for chunk in context_chunks[:top_k]])
    top_metadata = [chunk[1] for chunk in context_chunks[:top_k]]

    metadata_info = "\n".join([
        f"Doc {meta['doc_idx']} | Citation: {meta['citation_information']} | URL: {meta['url']} | Label: {meta['label']}"
        for meta in top_metadata
    ])

    prompt = f"""You are a legal assistant. Based on the following legal context and metadata, 
draft a precise and professional legal clause or document section.

Context:
{top_context}

Metadata:
{metadata_info}

Instruction:
{query}
"""

    response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
    return response.text

In [54]:
# ========================
# 7. Main Function
# ========================
def main():
    print("🔹 Loading and chunking data...")
    chunked_text, metadatas = load_and_chunk_data()

    print("🔹 Setting up vector database...")
    collection, local_model = setup_vector_db(chunked_text, metadatas)

    query = input("Enter your legal drafting instruction: ")
    print(f"🔹 Running hybrid search for query: {query}")
    candidate_chunks, candidate_metas = hybrid_search(query, collection, local_model)

    print("🔹 Re-ranking top candidates using Gemini + Local Embeddings...")
    reranked = rerank_chunks(query, candidate_chunks, candidate_metas, local_model)

    print("🔹 Generating legal draft...")
    legal_draft = generate_legal_draft(query, reranked)

    print("\n LEGAL DRAFT OUTPUT:")
    print(legal_draft)

In [ ]:
if __name__ == "__main__":
    main()

🔹 Loading and chunking data...
🔹 Setting up vector database...


Batches:   0%|          | 0/2853 [00:00<?, ?it/s]